# Chuyển tất cả file PDF sang folder riêng


In [ ]:
import os
import re
import io
import time
import fitz  # PyMuPDF để đếm số trang chính xác
from google.cloud import vision
from pdf2image import convert_from_path
import shutil
from concurrent.futures import ThreadPoolExecutor
import threading

In [ ]:
# ✅ Đặt đường dẫn Google Cloud Vision JSON Key
json_key_path = r"" #file json của m
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_key_path  

poppler_path = r"poppler-21.03.0\Library\bin"

# ✅ Sử dụng đa luồng để tăng tốc độ xử lý
workers = 30 #CPU 12600KF 10 nhân 16 luồng

In [ ]:
# ✅ Cấu hình thư mục trên máy tính
data_folder = r"Data"
input_folder = os.path.join(data_folder, "Input")  # Thư mục chứa PDF
txt_folder = os.path.join(data_folder, "Output", "Success TXT File")  # Thư mục lưu TXT
success_folder = os.path.join(data_folder, "Output", "Success File") #Thư mục lưu PDF convert thành công
failed_folder = os.path.join(data_folder, "Output", "Failed File") #Thư mục lưu PDF convert thất bại
downloads_folder = r"downloads"  # Thư mục gốc chứa tất cả các file pdf đã unzip

os.makedirs(input_folder, exist_ok=True)
os.makedirs(txt_folder, exist_ok=True)
os.makedirs(success_folder, exist_ok=True)
os.makedirs(failed_folder, exist_ok=True)

In [ ]:
import os
import shutil

def move_files_recursive(downloads_folder, input_folder):
    """Duyệt toàn bộ thư mục con và di chuyển tất cả các file sang input_folder"""
    if not os.path.exists(downloads_folder):
        print(f"❌ Thư mục không tồn tại: {downloads_folder}")
        return

    file_count = 0  # Đếm số file đã di chuyển
    for root, _, files in os.walk(downloads_folder):  # Duyệt tất cả thư mục con
        for file_name in files:
            source_path = os.path.join(root, file_name)
            destination_path = os.path.join(input_folder, file_name)

            # Xử lý trùng tên file (nếu có)
            counter = 1
            while os.path.exists(destination_path):
                file_base, file_ext = os.path.splitext(file_name)
                new_name = f"{file_base}_{counter}{file_ext}"
                destination_path = os.path.join(input_folder, new_name)
                counter += 1

            try:
                shutil.move(source_path, destination_path)
                file_count += 1
                print(f"✅ Moved: {file_name} → {destination_path}")
            except Exception as e:
                print(f"❌ Lỗi khi di chuyển {file_name}: {e}")

    print(f"🎉 Tổng số file đã di chuyển: {file_count}")

# Chạy chương trình
move_files_recursive(downloads_folder, input_folder)

# Xóa các đuôi file txt, pdf _1, _2, _3 vì đó là file trùng

Di chuyển File từ Success và Failed Folder về Input Folder (Chỉ chạy code này khi cần thiết)

In [ ]:
# def move_files_back(source_folder, destination_folder):
#     for file_name in os.listdir(source_folder):
#         source_path = os.path.join(source_folder, file_name)
#         destination_path = os.path.join(destination_folder, file_name)
#         shutil.move(source_path, destination_path)

# # Move files back to input folder
# move_files_back(success_folder, input_folder)
# move_files_back(failed_folder, input_folder)

# print("✅ All files moved back to input folder!")

In [ ]:
# ✅ Remove duplicate files across all types
deleted_files = []

for file in os.listdir(input_folder):
    if "_1." in file:  # Check for duplicate pattern (e.g., file_1.pdf, file_1.docx)
        original_file = file.replace("_1.", ".")
        original_path = os.path.join(input_folder, original_file)
        duplicate_path = os.path.join(input_folder, file)

        if os.path.exists(original_path):  # Delete only if original exists
            os.remove(duplicate_path)
            deleted_files.append(file)

# ✅ Display results
print(f"📂 Tổng số file đã xóa: {len(deleted_files)}")
if deleted_files:
    print("📌 Danh sách file bị xóa:")
    for f in deleted_files:
        print(f)

# ✅ Save log of deleted files
deleted_log_path = os.path.join(data_folder, "deleted_files.txt")
with open(deleted_log_path, "w", encoding="utf-8") as log_file:
    log_file.write("File bị xóa:\n" + "\n".join(deleted_files) + "\n")

print(f"📜 Danh sách file đã xóa được lưu tại: {deleted_log_path}")

# Đếm tổng số file PDF và DOC

In [ ]:
moved_pdf_files = len([f for f in os.listdir(input_folder) if f.lower().endswith(".pdf")])
print(f"📂 Tổng số file PDF còn: {moved_pdf_files}")

moved_doc_files = len([f for f in os.listdir(input_folder) if f.lower().endswith((".docx", ".doc"))])
print(f"📂 Tổng số file DOC/DOCX còn: {moved_doc_files}")

# GGVision OCR


In [ ]:
# ✅ Khởi tạo Google Vision Client
client = vision.ImageAnnotatorClient()

# ✅ Lấy danh sách tất cả file PDF
pdf_files = sorted([f for f in os.listdir(input_folder) if f.lower().endswith(".pdf")], 
                   key=lambda f: os.path.getsize(os.path.join(input_folder, f)))
total_files = len(pdf_files)

# ✅ Lấy danh sách file TXT đã có để tránh xử lý lại
existing_txt_files = {os.path.splitext(f)[0] for f in os.listdir(txt_folder) if f.endswith(".txt")}

# ✅ Lọc ra danh sách PDF cần xử lý
pdf_files_to_process = [f for f in pdf_files if os.path.splitext(f)[0] not in existing_txt_files]
total_files_to_process = len(pdf_files_to_process)

print(f"📂 Tổng số file PDF: {total_files}")
print(f"⚡ Số file cần xử lý: {total_files_to_process}")

# ✅ Khóa để đảm bảo thứ tự in ra đúng
print_lock = threading.Lock()

# ✅ Hàm OCR trực tiếp từ ảnh trong bộ nhớ
def extract_text_from_image(image, max_retries=3):
    try_count = 0
    while try_count < max_retries:
        try:
            image_byte_array = io.BytesIO()
            image.save(image_byte_array, format="PNG")  # Chuyển ảnh thành byte
            image_byte_array = image_byte_array.getvalue()

            vision_image = vision.Image(content=image_byte_array)
            response = client.document_text_detection(image=vision_image)

            if response.error.message:
                raise Exception(response.error.message)

            return response.full_text_annotation.text.strip() if response.full_text_annotation.text else ""

        except Exception as e:
            try_count += 1
            with print_lock:
                print(f"❌ Lỗi OCR ({try_count}/{max_retries}): {str(e)}")

            if "503" in str(e):  # Lỗi Google API quá tải
                with print_lock:
                    print("⏳ Đợi 5 giây rồi thử lại...")
                time.sleep(5)

            elif try_count == max_retries:
                with print_lock:
                    print("❌ Không thể xử lý ảnh này sau nhiều lần thử!")
                return ""

# ✅ Xử lý PDF với đa luồng
failed_files = []  # Danh sách file gặp lỗi

def process_pdf(file_name, index):
    pdf_path = os.path.join(input_folder, file_name)
    pdf_name = os.path.splitext(file_name)[0]  # Lấy tên file, bỏ đuôi .pdf
    txt_path = os.path.join(txt_folder, f"{pdf_name}.txt")

    # ✅ Kiểm tra tránh tạo file trùng
    if pdf_name in existing_txt_files:
        with print_lock:
            print(f"⚠️ Bỏ qua {file_name} (TXT đã tồn tại)")
        return

    with print_lock:
        print(f"\n📄 [{index+1}/{total_files_to_process}] Đang xử lý: {file_name}...")

    try:
        with fitz.open(pdf_path) as doc:
            total_pages = len(doc)
        
        with print_lock:
            print(f"📜 Tổng số trang: {total_pages}")

        extracted_texts = []
        for start_page in range(1, total_pages + 1, 50):
            end_page = min(start_page + 49, total_pages)
            with print_lock:
                print(f"🔄 Đang xử lý từ trang {start_page} đến {end_page}...")

            images = convert_from_path(pdf_path, first_page=start_page, last_page=end_page)  

            for page_number, image in enumerate(images, start=start_page):
                text = extract_text_from_image(image)
                time.sleep(1)  # Giảm tải API bằng cách chờ 1 giây

                if text:
                    extracted_texts.append(text)

        if extracted_texts:
            # ✅ Xóa file TXT nếu đã tồn tại (để tránh tạo file trùng)
            if os.path.exists(txt_path):
                os.remove(txt_path)

            # ✅ Ghi đè file TXT (thay vì append)
            with open(txt_path, "w", encoding="utf-8") as txt_file:
                txt_file.write("\n".join(extracted_texts) + "\n")

        shutil.move(pdf_path, os.path.join(success_folder, file_name))
    
    except Exception as e:
        with print_lock:
            print(f"❌ Lỗi khi xử lý PDF {file_name}: {str(e)}")
        failed_files.append(file_name)
        shutil.move(pdf_path, os.path.join(failed_folder, file_name))
        return

    remaining_files = total_files_to_process - (index + 1)
    with print_lock:
        print(f"✅ OCR hoàn thành: {file_name} -> {txt_path} ({remaining_files} file còn lại)")

# ✅ Chạy đa luồng
with ThreadPoolExecutor(max_workers=workers) as executor:
    executor.map(process_pdf, pdf_files_to_process, range(total_files_to_process))

print("\n🎉 Hoàn thành OCR cho tất cả file PDF mới! 🚀")

# ✅ Ghi danh sách file lỗi
if failed_files:
    failed_files_path = os.path.join(data_folder, "failed_files.txt")
    with open(failed_files_path, "w", encoding="utf-8") as log_file:
        log_file.write("\n".join(failed_files))
    print(f"⚠️ Một số file gặp lỗi. Danh sách tại: {failed_files_path}")

# ✅ Ghi danh sách file đã bỏ qua
skipped_files_path = os.path.join(data_folder, "skipped_files.txt")
with open(skipped_files_path, "w", encoding="utf-8") as log_file:
    log_file.write("\n".join(existing_txt_files))
print(f"📜 Danh sách file đã bỏ qua: {skipped_files_path}")